In [54]:
# from attr import dataclass
from dataclasses import dataclass, field
import swtoolkit as swtk
import win32com.client
import os
import csv




def check_file_exists(file_path):
    """Checks if a file exists. Returns True if it does, False if it doesn't."""
    if os.path.isfile(file_path):
        return 
    else:
        open(file_path, "w").close

def get_design_table_from_model_as_list(sw_model):
    """Returns the design table from a model. Returns None if no design table exists."""
    design_table = sw_model.GetDesignTable

    if design_table is None:
        return None
    else:
        dt = []
        nTotRow = design_table.GetTotalRowCount
        nTotCol = design_table.GetTotalColumnCount
        nTotRow = nTotRow + 1
        nTotCol = nTotCol + 1


        design_table.Attach

        # Generate Header Row
        header_row = []
        header_row.append("Config_Name")

        for c_index in range(1, nTotCol):
            header_row.append(design_table.GetHeaderText(c_index-1))

        dt.append(header_row)

        # Populate List of table row data
        for r_index in range(1, nTotRow):
            row_data = []
            for c_index in range(nTotCol):
                row_data.append(design_table.GetEntryValue(r_index, c_index))
            dt.append(row_data)    
        
        design_table.Detach

        return dt


@dataclass
class SW_Configuration:
    """Class for Solidworks Configurations - represents a row in a design table"""
    config_name: str
    header_columns: list
    row_data: list

    def update_config(self):
        """Updates the configuration values in Solidworks Design Table"""
        pass

    def add_config(self):
        """Adds a new configuration to the Solidworks Design Table"""
        pass


@dataclass
class SW_DesignTable:
    """Class for Solidworks Design Tables"""
    sw_model: win32com.client.CDispatch
    sw_template_dir: str 
    name: str = field(init=False)
    table: list = field(init=False)

    def __post_init__(self):
        self.name = self.sw_model.GetTitle
        self.table = self.get_design_table_from_model_as_list()

    def get_design_table_from_model_as_list(self):
        """Returns the design table from a model. Returns None if no design table exists."""
        design_table = self.sw_model.GetDesignTable
        if design_table is None:
            return None
        else:
            design_table.Attach

            # Generate Header Row
            header_row = []
            header_row.append("Config_Name")
            nTotRow = design_table.GetTotalRowCount + 1
            nTotCol = design_table.GetTotalColumnCount + 1
            for c_index in range(1, nTotCol):
                header_row.append(design_table.GetHeaderText(c_index-1))

            # Populate List of SW_Configuration objects
            configs = []
            for r_index in range(1, nTotRow):
                row_data = []
                for c_index in range(nTotCol):
                    row_data.append(design_table.GetEntryValue(r_index, c_index))
                configs.append(SW_Configuration(row_data[0], header_row, row_data[1:]))    

            design_table.Detach
            return configs

    def update_table(self):
        """Updates the configuration values in Solidworks Design Table"""
        design_table = self.sw_model.GetDesignTable()
        design_table.Attach()

        for r_idx, config_row in enumerate(self.table):
            for c_idx, value in enumerate(config_row.row_data):
                is_text = isinstance(value, str)
                design_table.SetEntryValue(r_idx, c_idx, is_text, value)
        
        design_table.UpdateTable(2, True)
        design_table.Detach()

    def add_config(self, config_row: SW_Configuration):
        """Adds a new configuration to the Solidworks Design Table"""
        design_table = self.sw_model.GetDesignTable
        # design_table.Attach
        cells = [config_row.config_name] + config_row.row_data
        print(f"      New Row Cells: {cells}")
        design_table.EditTable2(False)
        boolstatus = design_table.AddRow(cells)
        print(f"    AddRow successful?: {boolstatus}")
        boolstatus = design_table.UpdateTable(2, True)  # 2 corresponds to swUpdateDesignTableAll constant
        print(f"    UpdateTable successful?: {boolstatus}")
        # design_table.Detach

    def write_to_csv(self):
        """Writes the design table to a csv file with the same name as the model. Creates new CSV if doesnt exist, completely overwrites if it does."""
        csv_file_path = os.path.join(self.sw_template_dir, self.name + ".csv")
        try:
            with open(csv_file_path, 'w') as csv_file:
                for config in self.table:
                    csv_file.write(",".join([config.config_name] + config.row_data) + "\n")
        except FileNotFoundError:
            open(csv_file_path, "w").close()

    def update_from_csv(self):
        """Updates the design table from a CSV file"""
        print(f"\n\nUpdating design table from CSV file...\n")
        # Build the path to the CSV file
        csv_file_path = os.path.join(self.sw_template_dir, self.name + ".csv")

        # Read the CSV file
        with open(csv_file_path, 'r') as csv_file:
            reader = csv.reader(csv_file)
            header_row = next(reader)
            print(f"  Header Row: {header_row}")
            csv_configurations = []

            for row in reader:
                # Create a SW_Configuration for each row in the CSV file
                csv_configurations.append(SW_Configuration(row[0], header_row[1:], row[1:]))
                print(f"  CSV Config: {csv_configurations[-1]}")

        # Iterate over the configurations from the CSV file
        for csv_config in csv_configurations:
            # Check if this configuration is already in the design table
            in_table = False
            for dt_config in self.table:
                if dt_config.config_name == csv_config.config_name:
                    in_table = True
                    # The configuration is in the design table, so update it
                    dt_config.row_data = csv_config.row_data
                    # self.update_config(dt_config)
                    # break
            
            if not in_table:
                # The configuration is not in the design table, so add it
                self.add_config(csv_config)
        self.update_table()
            



# @dataclass
# class SW_Configuration:
#     """Class for Solidworks Configurations - represents a row in a design table"""
#     config_name: str
#     header_columns: list
#     row_data: list
#     # table: list = field(init=False)
#     def __post_init__(self):
#         # self.table = []
#         pass

#     def update_config(self):
#         """Updates the configuration values in Solidworks Design Table"""
#         pass

#     def add_config(self):
#         """Adds a new configuration to the Solidworks Design Table"""
#         pass


# @dataclass
# class SW_DesignTable:
#     """Class for Solidworks Design Tables"""
#     sw_model: win32com.client.CDispatch
#     sw_template_dir: str 
#     name: str = field(init=False)
#     table: list = field(init=False)

#     def __post_init__(self):
#         self.name = self.sw_model.GetTitle
#         self.table = get_design_table_from_model_as_list(self.sw_model)

#     def update_config(self, config_row: SW_Configuration):
#         """Updates the configuration values in Solidworks Design Table"""
#         pass

#     def add_config(self, config_row: SW_Configuration):
#         """Adds a new configuration to the Solidworks Design Table"""
#         pass

#     def write_to_csv(self): #, csv_file_loc=r'G:\My Drive\Google Drive - Work\TestMacros'):

#         # Check if file exists, if not create it
#         csv_file_path = os.path.join(self.sw_template_dir, self.name + ".csv")
#         check_file_exists(csv_file_path)

#         # Write table to csv
#         with open(csv_file_path, 'w') as csv_file:
#             for row in self.table:
#                 csv_file.write(",".join(row) + "\n")

#     def update_from_csv(self): #, csv_file_loc=r'G:\My Drive\Google Drive - Work\TestMacros'):
#         pass




# def write_design_table_to_csv(sw_model, csv_file_loc=r'G:\My Drive\Google Drive - Work\TestMacros'):
#     """Writes the design table to a csv file with the same name as the model. Creates new CSV if doesnt exist, completely overwrites if it does."""
#     # get design table
#     model_name = sw_model.GetTitle

#     design_table = get_design_table_from_model_as_list(sw_model)
    

#     nTotRow = len(design_table)
#     nTotCol = len(design_table[0])
#     print(f"Design Table Rows: {nTotRow}")
#     print(f"Design Table Columns: {nTotCol}")

#     csv_file_name = csv_file_loc + "\\" + model_name + ".csv"
#     print(f"CSV File Name: {csv_file_name}\n\n")

#     for row in design_table:
#         print(row)

#     with open(csv_file_name, 'w') as csv_file:
#         for r_index in range(nTotRow):
#             for c_index in range(nTotCol):
#                 csv_file.write(f"{str(design_table[r_index][c_index])}")
#                 if c_index < nTotCol - 1:
#                     csv_file.write(",")
#             csv_file.write("\n")

#     return

# def update_design_table_from_csv(sw_model, csv_file_dir=r'G:\My Drive\Google Drive - Work\TestMacros', csv_file_name):
#     """Updates the design table from a csv file. Creates new configuration if it does not exist in the design table."""
#     # get design table
#     model_name = sw_model.GetTitle
#     design_table = sw_model.GetDesignTable

#     return



# sw = win32com.client.Dispatch("SldWorks.Application")
sw = swtk.SolidWorks()

# get active document
model = sw._active_doc()
# model = sw.ActiveDoc
# write_design_table_to_csv(model)
# print(f"\n\n{model.GetTitle}\n\n")

dt = SW_DesignTable(model, r'G:\My Drive\Google Drive - Work\TestMacros')

dt.update_from_csv()

# # print(type(model))

# # get design table
# design_table = model.GetDesignTable

# # print(type(design_table))

# bRet = design_table.Attach

# print(bRet)

# nTotRow = design_table.GetTotalRowCount
# nTotCol = design_table.GetTotalColumnCount

# print(f"Number of Rows:    {nTotRow}")
# print(f"Number of Columns: {nTotCol}\n\n")

# nTotRow = nTotRow + 1
# nTotCol = nTotCol + 1

# for r_index in range(1, nTotRow):
#     for c_index in range(nTotCol):
#         print(f"Row: {r_index} Column: {c_index} | Column: {design_table.GetHeaderText(c_index-1)}   Value: {design_table.GetEntryValue(r_index, c_index)}")

# print(f"\nNow using Entry Text:\n")

# for r_index in range(1, nTotRow):
#     for c_index in range(nTotCol):
#         print(f"Row: {r_index} Column: {c_index} | Column: {design_table.GetHeaderText(c_index-1)}   Value: {design_table.GetEntryText(r_index, c_index)}")



# bRet = design_table.Detach

sw_model = sw.get_model()
sw_model.extension.rebuild(1)
print(f"\n\nDone.\n\n")





Updating design table from CSV file...

  Header Row: ['Config_Name', '$DESCRIPTION', '$COLOR', 'Horizontal_Dim@Panel_Sketch', 'Vertical_Dim@Panel_Sketch', 'Horizontal_Centre_Offset@Panel_Sketch', 'Vertical_Centre_Offset@Panel_Sketch', 'Panel_Thickness_Dim@Panel_Thickness']
  CSV Config: SW_Configuration(config_name='Default', header_columns=['$DESCRIPTION', '$COLOR', 'Horizontal_Dim@Panel_Sketch', 'Vertical_Dim@Panel_Sketch', 'Horizontal_Centre_Offset@Panel_Sketch', 'Vertical_Centre_Offset@Panel_Sketch', 'Panel_Thickness_Dim@Panel_Thickness'], row_data=['Default', '15651274.0', '4.3885071', '4.0', '0.0', '0.0', '0.7500000000000001'])
  CSV Config: SW_Configuration(config_name='Test1', header_columns=['$DESCRIPTION', '$COLOR', 'Horizontal_Dim@Panel_Sketch', 'Vertical_Dim@Panel_Sketch', 'Horizontal_Centre_Offset@Panel_Sketch', 'Vertical_Centre_Offset@Panel_Sketch', 'Panel_Thickness_Dim@Panel_Thickness'], row_data=['Test1', '15651274.0', '5.0', '3.0000000000000004', '0.1', '0.1', '1.0'

com_error: (-2147417851, 'The server threw an exception.', None, None)